<a href="https://colab.research.google.com/github/ArijaK/QuestionAnswering/blob/main/QA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>